In [1]:
from elasticsearch import Elasticsearch
host = 'http://localhost:9201'

es = Elasticsearch(host)



/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
mapping = {
    "mappings": {
        "properties": {
            "pages": {
                "type": "integer"
            }
            
        }
    }
}

es.indices.create(index="new_books_info_index", body=mapping)

/tmp/ipykernel_6497/1066748123.py:13: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index="new_books_info_index", body=mapping)
/tmp/ipykernel_6497/1066748123.py:13: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index="new_books_info_index", body=mapping)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'new_books_info_index'})

In [3]:
reindex_body = {
    "source": {
        "index": "books_info_index"
    },
    "dest": {
        "index": "new_books_info_index"
    }
}

es.reindex(body=reindex_body)

/tmp/ipykernel_6497/493192152.py:11: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.reindex(body=reindex_body)
/tmp/ipykernel_6497/493192152.py:11: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.reindex(body=reindex_body)


ObjectApiResponse({'took': 5276, 'timed_out': False, 'total': 42174, 'updated': 0, 'created': 42174, 'deleted': 0, 'batches': 43, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [4]:
query = {
    "query": {
        "bool": {
            "must": {
                "exists": {
                    "field": "children"
                }
            },
            "must_not": {
                "term": {
                    "children.keyword": ""
                }
            }
        }
    }
}

response = es.search(index="new_books_info_index", body=query)

result_count = response['hits']['total']['value']
print("Number of Results:", result_count)

for hit in response['hits']['hits']:
    print(hit['_source'])

/tmp/ipykernel_6497/2380763676.py:19: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index="new_books_info_index", body=query)


Number of Results: 240
{'title': 'A Malay-English dictionary (Romanised) / by R.J. Wilkinson', 'url': 'https://trove.nla.gov.au/work/10293327', 'contributors': 'Wilkinson, R. J. (Richard James), 1867-1941', 'date': '1932-1969', 'format': 'Book', 'fulltext_url': 'http://nla.gov.au/nla.obj-60272760', 'trove_id': 'nla.obj-60272760', 'language': '', 'rights': '', 'pages': '0', 'form': 'Multi Volume Book', 'volume': '', 'parent': '', 'children': 'nla.obj-60272771|nla.obj-60272783', 'text_downloaded': 'False', 'text_file': ''}
{'title': "The Plus in your policy / issued by the leading life asurance offices in Australia and sent to you with the compliments of the Mutual Life & Citizens' Assurance Co. Ltd", 'url': 'https://trove.nla.gov.au/work/10396117', 'contributors': 'MLC', 'date': '1949', 'format': 'Book', 'fulltext_url': 'https://nla.gov.au/nla.obj-2428558076', 'trove_id': 'nla.obj-2428558076', 'language': 'English', 'rights': 'Out of Copyright|http://rightsstatements.org/vocab/NKC/1.0/'

/tmp/ipykernel_6497/2380763676.py:19: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.search(index="new_books_info_index", body=query)


In [5]:

query = {
    "query": {
        "bool": {
            "must": [
                {
                    "exists": {
                        "field": "children"
                    }
                },
                {
                    "bool": {
                        "must_not": [
                            {
                                "term": {
                                    "children.keyword": ""
                                }
                            }
                        ]
                    }
                }
            ]
        }
    },
    "aggs": {
        "avg_pages": {
            "avg": {
                "field": "pages"
            }
        }
    }
}


response = es.search(index="new_books_info_index", body=query)

for hit in response['hits']['hits']:
    print(hit['_source'])

avg_pages = response['aggregations']['avg_pages']['value']
print("Average Pages:", avg_pages)


{'title': 'A Malay-English dictionary (Romanised) / by R.J. Wilkinson', 'url': 'https://trove.nla.gov.au/work/10293327', 'contributors': 'Wilkinson, R. J. (Richard James), 1867-1941', 'date': '1932-1969', 'format': 'Book', 'fulltext_url': 'http://nla.gov.au/nla.obj-60272760', 'trove_id': 'nla.obj-60272760', 'language': '', 'rights': '', 'pages': '0', 'form': 'Multi Volume Book', 'volume': '', 'parent': '', 'children': 'nla.obj-60272771|nla.obj-60272783', 'text_downloaded': 'False', 'text_file': ''}
{'title': "The Plus in your policy / issued by the leading life asurance offices in Australia and sent to you with the compliments of the Mutual Life & Citizens' Assurance Co. Ltd", 'url': 'https://trove.nla.gov.au/work/10396117', 'contributors': 'MLC', 'date': '1949', 'format': 'Book', 'fulltext_url': 'https://nla.gov.au/nla.obj-2428558076', 'trove_id': 'nla.obj-2428558076', 'language': 'English', 'rights': 'Out of Copyright|http://rightsstatements.org/vocab/NKC/1.0/', 'pages': '0', 'form':

/tmp/ipykernel_6497/285560300.py:34: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index="new_books_info_index", body=query)
/tmp/ipykernel_6497/285560300.py:34: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.search(index="new_books_info_index", body=query)


In [6]:

query = {
    "query": {
        "bool": {
            "must": [
                {
                    "exists": {
                        "field": "parent"
                    }
                },
                {
                    "bool": {
                        "must_not": [
                            {
                                "term": {
                                    "parent.keyword": ""
                                }
                            }
                        ]
                    }
                }
            ]
        }
    },
    "aggs": {
        "avg_pages": {
            "avg": {
                "field": "pages"
            }
        }
    }
}


response = es.search(index="new_books_info_index", body=query)


for hit in response['hits']['hits']:
    print(hit['_source'])

avg_pages = response['aggregations']['avg_pages']['value']
print("Average :", avg_pages)


{'title': 'A Malay-English dictionary (Romanised) / by R.J. Wilkinson', 'url': 'https://trove.nla.gov.au/work/10293327', 'contributors': 'Wilkinson, R. J. (Richard James), 1867-1941', 'date': '1932-1969', 'format': 'Book', 'fulltext_url': 'http://nla.gov.au/nla.obj-60272771', 'trove_id': 'nla.obj-60272771', 'language': '', 'rights': '', 'pages': '644', 'form': 'Multi Volume Book', 'volume': '1', 'parent': 'nla.obj-60272760', 'children': '', 'text_downloaded': 'True', 'text_file': 'a-malay-english-dictionary-romanised-by-r-j-w-nla.obj-60272771.txt'}
{'title': 'A Malay-English dictionary (Romanised) / by R.J. Wilkinson', 'url': 'https://trove.nla.gov.au/work/10293327', 'contributors': 'Wilkinson, R. J. (Richard James), 1867-1941', 'date': '1932-1969', 'format': 'Book', 'fulltext_url': 'http://nla.gov.au/nla.obj-60272783', 'trove_id': 'nla.obj-60272783', 'language': '', 'rights': '', 'pages': '674', 'form': 'Multi Volume Book', 'volume': '2', 'parent': 'nla.obj-60272760', 'children': '', 

/tmp/ipykernel_6497/14478565.py:35: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index="new_books_info_index", body=query)
/tmp/ipykernel_6497/14478565.py:35: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.search(index="new_books_info_index", body=query)


In [8]:
client = Elasticsearch(host)
search_body = {
    "size": 0,
    "aggs": {
        "formats": {
            "terms": {
                "field": "date.keyword",
                "size": 43000 
            }
        }
    }
}

response = client.search(index="books_info_index", body=search_body)

aggregations = response["aggregations"]
formats_buckets = aggregations["formats"]["buckets"]
books_dates ={}
for bucket in formats_buckets:
    format_value = bucket["key"]
    doc_count = bucket["doc_count"]
    books_dates[format_value] = doc_count
    print(f"Format: {format_value}, Count: {doc_count}")

Format: 2018, Count: 1594
Format: 2017, Count: 1378
Format: 2016, Count: 1085
Format: 1900, Count: 777
Format: 2015, Count: 770
Format: 2019, Count: 725
Format: 1716, Count: 606
Format: 2012, Count: 598
Format: 2014, Count: 503
Format: 2013, Count: 493
Format: 2011, Count: 468
Format: 2009, Count: 431
Format: 1915, Count: 424
Format: 2000, Count: 402
Format: 2008, Count: 400
Format: 2010, Count: 378
Format: 2007, Count: 355
Format: 1918, Count: 335
Format: 1916, Count: 315
Format: 2006, Count: 312
Format: 2005, Count: 290
Format: 1917, Count: 284
Format: 2004, Count: 271
Format: 1919, Count: 268
Format: 1907, Count: 266
Format: 1888, Count: 263
Format: 2003, Count: 247
Format: 1886, Count: 242
Format: 2002, Count: 241
Format: 1914, Count: 236
Format: 1992, Count: 233
Format: 1990, Count: 232
Format: 1978, Count: 231
Format: 1991, Count: 231
Format: 1977, Count: 230
Format: 2001, Count: 226
Format: 1934, Count: 221
Format: 1997, Count: 217
Format: 1910, Count: 212
Format: 1986, Count: 2

/tmp/ipykernel_6497/2466289000.py:15: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index="books_info_index", body=search_body)
/tmp/ipykernel_6497/2466289000.py:15: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = client.search(index="books_info_index", body=search_body)


In [9]:
books_dates_max = max(books_dates,key = books_dates.get)
print(books_dates_max)

2018


In [14]:
query = {
    "_source": ["trove_id", "title", "date", "pages", "parent", "format", "url"],
    "query": {
        "bool": {
            "must": [
                {
                    "exists": {
                        "field": "parent"
                    }
                },
                {
                    "bool": {
                        "must_not": [
                            {
                                "term": {
                                    "parent.keyword": ""
                                }
                            }
                        ]
                    }
                }
            ]
        }
    }
}

results = es.search(index="new_books_info_index", body=query)

for hit in results["hits"]["hits"]:
    print(hit["_source"])


{'date': '1932-1969', 'parent': 'nla.obj-60272760', 'pages': '644', 'format': 'Book', 'title': 'A Malay-English dictionary (Romanised) / by R.J. Wilkinson', 'trove_id': 'nla.obj-60272771', 'url': 'https://trove.nla.gov.au/work/10293327'}
{'date': '1932-1969', 'parent': 'nla.obj-60272760', 'pages': '674', 'format': 'Book', 'title': 'A Malay-English dictionary (Romanised) / by R.J. Wilkinson', 'trove_id': 'nla.obj-60272783', 'url': 'https://trove.nla.gov.au/work/10293327'}
{'date': '1949', 'parent': 'nla.obj-2428558076', 'pages': '1', 'format': 'Book', 'title': "The Plus in your policy / issued by the leading life asurance offices in Australia and sent to you with the compliments of the Mutual Life & Citizens' Assurance Co. Ltd", 'trove_id': 'nla.obj-2438504941', 'url': 'https://trove.nla.gov.au/work/10396117'}
{'date': '1949', 'parent': 'nla.obj-2428558076', 'pages': '6', 'format': 'Book', 'title': "The Plus in your policy / issued by the leading life asurance offices in Australia and s

/tmp/ipykernel_6497/3119104452.py:27: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  results = es.search(index="new_books_info_index", body=query)
/tmp/ipykernel_6497/3119104452.py:27: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  results = es.search(index="new_books_info_index", body=query)


In [15]:
query = {
    "_source": ["trove_id", "title", "date", "pages", "children", "format", "url"],
    "query": {
        "bool": {
            "must": [
                {
                    "exists": {
                        "field": "children"
                    }
                },
                {
                    "bool": {
                        "must_not": [
                            {
                                "term": {
                                    "children.keyword": ""
                                }
                            }
                        ]
                    }
                }
            ]
        }
    }
}

results = es.search(index="new_books_info_index", body=query)

for hit in results["hits"]["hits"]:
    print(hit["_source"])


{'date': '1932-1969', 'pages': '0', 'children': 'nla.obj-60272771|nla.obj-60272783', 'format': 'Book', 'title': 'A Malay-English dictionary (Romanised) / by R.J. Wilkinson', 'trove_id': 'nla.obj-60272760', 'url': 'https://trove.nla.gov.au/work/10293327'}
{'date': '1949', 'pages': '0', 'children': 'nla.obj-2438504941|nla.obj-2438506988', 'format': 'Book', 'title': "The Plus in your policy / issued by the leading life asurance offices in Australia and sent to you with the compliments of the Mutual Life & Citizens' Assurance Co. Ltd", 'trove_id': 'nla.obj-2428558076', 'url': 'https://trove.nla.gov.au/work/10396117'}
{'date': '1941-2004', 'pages': '0', 'children': 'nla.obj-77324219', 'format': 'Book|Book/Illustrated|Unpublished', 'title': "Henderson's Australian families : a genealogical and biographical record / compiled and edited by Alexander Henderson", 'trove_id': 'nla.obj-77324653', 'url': 'https://trove.nla.gov.au/work/11037485'}
{'date': '1857', 'pages': '0', 'children': 'nla.obj-4

/tmp/ipykernel_6497/3110956242.py:27: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  results = es.search(index="new_books_info_index", body=query)
/tmp/ipykernel_6497/3110956242.py:27: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  results = es.search(index="new_books_info_index", body=query)


In [72]:
from pyspark.sql import SparkSession
import json

query = {
    "size": 4000,
    "_source": ["trove_id", "title", "date", "pages", "parent", "format", "url"],
    "query": {
        "bool": {
            "must": [
                {
                    "exists": {
                        "field": "parent"
                    }
                },
                {
                    "bool": {
                        "must_not": [
                            {
                                "term": {
                                    "parent.keyword": ""
                                }
                            }
                        ]
                    }
                }
            ]
        }
    }
}

results = es.search(index="new_books_info_index", body=query)

spark = SparkSession.builder.appName("Elasticsearch to PySpark").getOrCreate()

data = list(map(lambda x: x["_source"], results["hits"]["hits"]))
rdd = spark.sparkContext.parallelize(data)
df_parent = spark.read.json(rdd)

df_parent.show()

/tmp/ipykernel_6497/915019976.py:36: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  results = es.search(index="new_books_info_index", body=query)
/tmp/ipykernel_6497/915019976.py:36: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  results = es.search(index="new_books_info_index", body=query)


+---------+--------------------+-----+------------------+--------------------+------------------+--------------------+
|     date|              format|pages|            parent|               title|          trove_id|                 url|
+---------+--------------------+-----+------------------+--------------------+------------------+--------------------+
|1932-1969|                Book|  644|  nla.obj-60272760|A Malay-English d...|  nla.obj-60272771|https://trove.nla...|
|1932-1969|                Book|  674|  nla.obj-60272760|A Malay-English d...|  nla.obj-60272783|https://trove.nla...|
|     1949|                Book|    1|nla.obj-2428558076|The Plus in your ...|nla.obj-2438504941|https://trove.nla...|
|     1949|                Book|    6|nla.obj-2428558076|The Plus in your ...|nla.obj-2438506988|https://trove.nla...|
|1941-2004|Book|Book/Illustr...|  453|  nla.obj-77324653|Henderson's Austr...|  nla.obj-77324219|https://trove.nla...|
|     1857|Book|Article/Othe...|  104| nla.obj-4

In [73]:
import pandas as pd

In [74]:
df_parent2 = df_parent

In [75]:
dff_aprent = df_parent.toPandas()

In [76]:
dff_aprent

,date,format,pages,parent,title,trove_id,url
0,1932-1969,Book,644,nla.obj-60272760,A Malay-English dictionary (Romanised) / by R....,nla.obj-60272771,https://trove.nla.gov.au/work/10293327
1,1932-1969,Book,674,nla.obj-60272760,A Malay-English dictionary (Romanised) / by R....,nla.obj-60272783,https://trove.nla.gov.au/work/10293327
2,1949,Book,1,nla.obj-2428558076,The Plus in your policy / issued by the leadin...,nla.obj-2438504941,https://trove.nla.gov.au/work/10396117
3,1949,Book,6,nla.obj-2428558076,The Plus in your policy / issued by the leadin...,nla.obj-2438506988,https://trove.nla.gov.au/work/10396117
4,1941-2004,Book|Book/Illustrated|Unpublished,453,nla.obj-77324653,Henderson's Australian families : a genealogic...,nla.obj-77324219,https://trove.nla.gov.au/work/11037485
...,...,...,...,...,...,...,...
1418,1907-1978,Book/Illustrated|Book,1124,nla.obj-63407750,The cyclopedia of South Australia in two volum...,nla.obj-74680162,https://trove.nla.gov.au/work/9554744
1419,1978,Book|Government publication|Article,288,nla.obj-1335763041,Review of Australia's international civil avia...,nla.obj-1351153218,https://trove.nla.gov.au/work/9574934
1420,1978,Book|Government publication|Article,448,nla.obj-1335763041,Review of Australia's international civil avia...,nla.obj-1351153613,https://trove.nla.gov.au/work/9574934
1421,1867-1873,Book|Government publication|Book/Illustrated,108,nla.obj-396862927,The progress of Victoria : a statistical essay...,nla.obj-398086951,https://trove.nla.gov.au/work/9800197


In [77]:
from pyspark.sql import SparkSession
import json

query = {
    "size": 4000,
    "_source": ["trove_id", "title", "date", "pages", "children", "format", "url"],
    "query": {
        "bool": {
            "must": [
                {
                    "exists": {
                        "field": "children"
                    }
                },
                {
                    "bool": {
                        "must_not": [
                            {
                                "term": {
                                    "children.keyword": ""
                                }
                            }
                        ]
                    }
                }
            ]
        }
    }
}

resultsch = es.search(index="new_books_info_index", body=query)

spark = SparkSession.builder.appName("Elasticsearch to PySpark").getOrCreate()

datach = list(map(lambda x: x["_source"], resultsch["hits"]["hits"]))
rddch = spark.sparkContext.parallelize(datach)
df_child = spark.read.json(rddch)

df_child.show()

/tmp/ipykernel_6497/3957242074.py:36: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resultsch = es.search(index="new_books_info_index", body=query)
/tmp/ipykernel_6497/3957242074.py:36: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  resultsch = es.search(index="new_books_info_index", body=query)


+--------------------+---------+--------------------+-----+--------------------+------------------+--------------------+
|            children|     date|              format|pages|               title|          trove_id|                 url|
+--------------------+---------+--------------------+-----+--------------------+------------------+--------------------+
|nla.obj-60272771|...|1932-1969|                Book|    0|A Malay-English d...|  nla.obj-60272760|https://trove.nla...|
|nla.obj-243850494...|     1949|                Book|    0|The Plus in your ...|nla.obj-2428558076|https://trove.nla...|
|    nla.obj-77324219|1941-2004|Book|Book/Illustr...|    0|Henderson's Austr...|  nla.obj-77324653|https://trove.nla...|
|nla.obj-412255342...|     1857|Book|Article/Othe...|    0|Abstract and anal...| nla.obj-412253504|https://trove.nla...|
|nla.obj-396883876...|     1861|Book/Illustrated|...|    0|The Burke and Wil...| nla.obj-396866331|https://trove.nla...|
|nla.obj-375325055...|1919-2003|

In [78]:
df_child2 = df_child

In [79]:

dff_child = df_child.toPandas()
dff_child

,children,date,format,pages,title,trove_id,url
0,nla.obj-60272771|nla.obj-60272783,1932-1969,Book,0,A Malay-English dictionary (Romanised) / by R....,nla.obj-60272760,https://trove.nla.gov.au/work/10293327
1,nla.obj-2438504941|nla.obj-2438506988,1949,Book,0,The Plus in your policy / issued by the leadin...,nla.obj-2428558076,https://trove.nla.gov.au/work/10396117
2,nla.obj-77324219,1941-2004,Book|Book/Illustrated|Unpublished,0,Henderson's Australian families : a genealogic...,nla.obj-77324653,https://trove.nla.gov.au/work/11037485
3,nla.obj-412255342|nla.obj-75124485,1857,Book|Article/Other article|Article,0,Abstract and analysis of the evidence taken by...,nla.obj-412253504,https://trove.nla.gov.au/work/11179694
4,nla.obj-396883876|nla.obj-61875251,1861,Book/Illustrated|Book,0,The Burke and Wills exploring expedition : an ...,nla.obj-396866331,https://trove.nla.gov.au/work/11299937
...,...,...,...,...,...,...,...
235,nla.obj-1194416238|nla.obj-1194416440|nla.obj-...,1997,Book|Book/Illustrated|Article,0,Five years on : implementation of the Commonwe...,nla.obj-1297647186,https://trove.nla.gov.au/work/8482284
236,nla.obj-1385727717|nla.obj-1385745492|nla.obj-...,1971,Book,0,Izesuq oko moloko imolaqu,nla.obj-1385695208,https://trove.nla.gov.au/work/9389532
237,nla.obj-63407775|nla.obj-74680162,1907-1978,Book/Illustrated|Book,0,The cyclopedia of South Australia in two volum...,nla.obj-63407750,https://trove.nla.gov.au/work/9554744
238,nla.obj-1351153218|nla.obj-1351153613,1978,Book|Government publication|Article,0,Review of Australia's international civil avia...,nla.obj-1335763041,https://trove.nla.gov.au/work/9574934


In [80]:
dff_aprent.columns

Index(['date', 'format', 'pages', 'parent', 'title', 'trove_id', 'url'], dtype='object')

In [81]:
from pyspark.sql import functions as F


In [82]:
df_parent = df_parent.withColumn("date_start", F.when(F.size(F.split("date", "-")) == 2, F.split("date", "-")[0]).otherwise(F.col("date")))
df_parent = df_parent.withColumn("date_finish", F.when(F.size(F.split("date", "-")) == 2, F.split("date", "-")[1]).otherwise(F.col("date")))

df_parent.show()

+---------+--------------------+-----+------------------+--------------------+------------------+--------------------+----------+-----------+
|     date|              format|pages|            parent|               title|          trove_id|                 url|date_start|date_finish|
+---------+--------------------+-----+------------------+--------------------+------------------+--------------------+----------+-----------+
|1932-1969|                Book|  644|  nla.obj-60272760|A Malay-English d...|  nla.obj-60272771|https://trove.nla...|      1932|       1969|
|1932-1969|                Book|  674|  nla.obj-60272760|A Malay-English d...|  nla.obj-60272783|https://trove.nla...|      1932|       1969|
|     1949|                Book|    1|nla.obj-2428558076|The Plus in your ...|nla.obj-2438504941|https://trove.nla...|      1949|       1949|
|     1949|                Book|    6|nla.obj-2428558076|The Plus in your ...|nla.obj-2438506988|https://trove.nla...|      1949|       1949|
|1941-

In [83]:
df_parent= df_parent.drop('date')

In [84]:
df_parent_pd = df_parent.toPandas()
df_parent_pd

,format,pages,parent,title,trove_id,url,date_start,date_finish
0,Book,644,nla.obj-60272760,A Malay-English dictionary (Romanised) / by R....,nla.obj-60272771,https://trove.nla.gov.au/work/10293327,1932,1969
1,Book,674,nla.obj-60272760,A Malay-English dictionary (Romanised) / by R....,nla.obj-60272783,https://trove.nla.gov.au/work/10293327,1932,1969
2,Book,1,nla.obj-2428558076,The Plus in your policy / issued by the leadin...,nla.obj-2438504941,https://trove.nla.gov.au/work/10396117,1949,1949
3,Book,6,nla.obj-2428558076,The Plus in your policy / issued by the leadin...,nla.obj-2438506988,https://trove.nla.gov.au/work/10396117,1949,1949
4,Book|Book/Illustrated|Unpublished,453,nla.obj-77324653,Henderson's Australian families : a genealogic...,nla.obj-77324219,https://trove.nla.gov.au/work/11037485,1941,2004
...,...,...,...,...,...,...,...,...
1418,Book/Illustrated|Book,1124,nla.obj-63407750,The cyclopedia of South Australia in two volum...,nla.obj-74680162,https://trove.nla.gov.au/work/9554744,1907,1978
1419,Book|Government publication|Article,288,nla.obj-1335763041,Review of Australia's international civil avia...,nla.obj-1351153218,https://trove.nla.gov.au/work/9574934,1978,1978
1420,Book|Government publication|Article,448,nla.obj-1335763041,Review of Australia's international civil avia...,nla.obj-1351153613,https://trove.nla.gov.au/work/9574934,1978,1978
1421,Book|Government publication|Book/Illustrated,108,nla.obj-396862927,The progress of Victoria : a statistical essay...,nla.obj-398086951,https://trove.nla.gov.au/work/9800197,1867,1873


In [85]:
df_child = df_child.withColumn("date_start", F.when(F.size(F.split("date", "-")) == 2, F.split("date", "-")[0]).otherwise(F.col("date")))
df_child = df_child.withColumn("date_finish", F.when(F.size(F.split("date", "-")) == 2, F.split("date", "-")[1]).otherwise(F.col("date")))

df_child.show()

+--------------------+---------+--------------------+-----+--------------------+------------------+--------------------+----------+-----------+
|            children|     date|              format|pages|               title|          trove_id|                 url|date_start|date_finish|
+--------------------+---------+--------------------+-----+--------------------+------------------+--------------------+----------+-----------+
|nla.obj-60272771|...|1932-1969|                Book|    0|A Malay-English d...|  nla.obj-60272760|https://trove.nla...|      1932|       1969|
|nla.obj-243850494...|     1949|                Book|    0|The Plus in your ...|nla.obj-2428558076|https://trove.nla...|      1949|       1949|
|    nla.obj-77324219|1941-2004|Book|Book/Illustr...|    0|Henderson's Austr...|  nla.obj-77324653|https://trove.nla...|      1941|       2004|
|nla.obj-412255342...|     1857|Book|Article/Othe...|    0|Abstract and anal...| nla.obj-412253504|https://trove.nla...|      1857|     

In [86]:
df_child= df_child.drop('date')

In [87]:
df_child_pd=df_child.toPandas()
df_child_pd

,children,format,pages,title,trove_id,url,date_start,date_finish
0,nla.obj-60272771|nla.obj-60272783,Book,0,A Malay-English dictionary (Romanised) / by R....,nla.obj-60272760,https://trove.nla.gov.au/work/10293327,1932,1969
1,nla.obj-2438504941|nla.obj-2438506988,Book,0,The Plus in your policy / issued by the leadin...,nla.obj-2428558076,https://trove.nla.gov.au/work/10396117,1949,1949
2,nla.obj-77324219,Book|Book/Illustrated|Unpublished,0,Henderson's Australian families : a genealogic...,nla.obj-77324653,https://trove.nla.gov.au/work/11037485,1941,2004
3,nla.obj-412255342|nla.obj-75124485,Book|Article/Other article|Article,0,Abstract and analysis of the evidence taken by...,nla.obj-412253504,https://trove.nla.gov.au/work/11179694,1857,1857
4,nla.obj-396883876|nla.obj-61875251,Book/Illustrated|Book,0,The Burke and Wills exploring expedition : an ...,nla.obj-396866331,https://trove.nla.gov.au/work/11299937,1861,1861
...,...,...,...,...,...,...,...,...
235,nla.obj-1194416238|nla.obj-1194416440|nla.obj-...,Book|Book/Illustrated|Article,0,Five years on : implementation of the Commonwe...,nla.obj-1297647186,https://trove.nla.gov.au/work/8482284,1997,1997
236,nla.obj-1385727717|nla.obj-1385745492|nla.obj-...,Book,0,Izesuq oko moloko imolaqu,nla.obj-1385695208,https://trove.nla.gov.au/work/9389532,1971,1971
237,nla.obj-63407775|nla.obj-74680162,Book/Illustrated|Book,0,The cyclopedia of South Australia in two volum...,nla.obj-63407750,https://trove.nla.gov.au/work/9554744,1907,1978
238,nla.obj-1351153218|nla.obj-1351153613,Book|Government publication|Article,0,Review of Australia's international civil avia...,nla.obj-1335763041,https://trove.nla.gov.au/work/9574934,1978,1978


In [88]:
df_child.describe()

DataFrame[summary: string, children: string, format: string, pages: string, title: string, trove_id: string, url: string, date_start: string, date_finish: string]

In [89]:
from pyspark.sql.functions import col

In [90]:
df_child = df_child.withColumn("date_start", col("date_start").cast("integer"))
df_child = df_child.withColumn("date_finish", col("date_finish").cast("integer"))
df_child = df_child.withColumn("pages", col("pages").cast("integer"))


In [91]:
df_parent = df_parent.withColumn("date_start", col("date_start").cast("integer"))
df_parent = df_parent.withColumn("date_finish", col("date_finish").cast("integer"))
df_parent = df_parent.withColumn("pages", col("pages").cast("integer"))


In [92]:
df_child.describe()

DataFrame[summary: string, children: string, format: string, pages: string, title: string, trove_id: string, url: string, date_start: string, date_finish: string]

In [93]:
df_child.columns

['children',
 'format',
 'pages',
 'title',
 'trove_id',
 'url',
 'date_start',
 'date_finish']

In [94]:
df_parent.columns

['format',
 'pages',
 'parent',
 'title',
 'trove_id',
 'url',
 'date_start',
 'date_finish']

In [95]:
df_child.printSchema()

root
 |-- children: string (nullable = true)
 |-- format: string (nullable = true)
 |-- pages: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- trove_id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- date_start: integer (nullable = true)
 |-- date_finish: integer (nullable = true)



In [96]:
df_parent.printSchema()

root
 |-- format: string (nullable = true)
 |-- pages: integer (nullable = true)
 |-- parent: string (nullable = true)
 |-- title: string (nullable = true)
 |-- trove_id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- date_start: integer (nullable = true)
 |-- date_finish: integer (nullable = true)



In [97]:
df_child.count()

240

In [98]:
df_parent.count()

1423

In [121]:
from cassandra.cluster import Cluster
import uuid

cluster = Cluster(["localhost"])
session = cluster.connect()

session.execute("CREATE KEYSPACE IF NOT EXISTS books_keyspace WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}")
session.execute("CREATE TABLE IF NOT EXISTS books_keyspace.children (id uuid , children TEXT,format TEXT,pages FLOAT,title TEXT,trove_id TEXT,url TEXT,date_start FLOAT,date_finish FLOAT,PRIMARY KEY((trove_id),id))")



In [122]:
from cassandra.query import SimpleStatement

insert_stmt = session.prepare("INSERT INTO books_keyspace.children (id, children, format, pages, title, trove_id, url, date_start, date_finish) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)")

for row in df_child.rdd.collect():
    row_id = uuid.uuid1()
    session.execute(insert_stmt, [row_id, row.children, row.format, row.pages, row.title, row.trove_id, row.url, row.date_start, row.date_finish])


In [105]:
session.execute("CREATE TABLE IF NOT EXISTS books_keyspace.parent (id uuid , parent TEXT,format TEXT,pages FLOAT,title TEXT,trove_id TEXT,url TEXT,date_start FLOAT,date_finish FLOAT,PRIMARY KEY((trove_id),id))")



In [118]:
from cassandra.query import SimpleStatement

insert_stmt2 = session.prepare("INSERT INTO books_keyspace.parent (id, parent, format, pages, title, trove_id, url, date_start, date_finish) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)")

for row in df_parent.rdd.collect():
    row_id = uuid.uuid1()
    
    session.execute(insert_stmt2, [row_id, row.parent, row.format, row.pages, row.title, row.trove_id, row.url, row.date_start, row.date_finish])


In [144]:
ro = session.execute("SELECT * FROM books_keyspace.children where date_start<2000 and date_finish<2000  ALLOW FILTERING" )
l = []
for row in ro:
    l.append((row.title, row.trove_id, row.url, row.date_start, row.date_finish,row.children, row.format,row.pages) )
    


[('Lists of men-of-war, 1650-1700',
  'nla.obj-33236945',
  'https://trove.nla.gov.au/work/13097671',
  1935.0,
  1938.0,
  'nla.obj-33239056',
  'Book|Book/Illustrated',
  0.0),
 ('[Bessie Campbell, Banjo Queen : a collection of music programmes, ephemera relating to her career]',
  'nla.obj-39267340',
  'https://trove.nla.gov.au/work/14080449',
  1889.0,
  1918.0,
  'nla.obj-1496267881|nla.obj-1496153359',
  'Book',
  0.0),
 ('[Bessie Campbell, Banjo Queen : a collection of music programmes, ephemera relating to her career]',
  'nla.obj-39267340',
  'https://trove.nla.gov.au/work/237278882',
  1889.0,
  1889.0,
  'nla.obj-1496267881|nla.obj-1496153359',
  'Book',
  0.0),
 ('Report of the proceedings of the Queensland Government schooner "Spitfire" in search of the mouth of the River Burdekin, on the north-eastern coast of Australia : and of the exploration of a portion of that coast extending from Gloucester Island to Halifax Bay',
  'nla.obj-475664029',
  'https://trove.nla.gov.au/w

In [147]:


rdd_selected_ch = spark.sparkContext.parallelize(l)

df_selected_ch = rdd_selected_ch.toDF(['title','trove_id',
 'url',
 'date_start',
 'date_finish',
    
    'children',
    'format',
    'pages'])

df_selected_ch.show()

+--------------------+------------------+--------------------+----------+-----------+--------------------+--------------------+-----+
|               title|          trove_id|                 url|date_start|date_finish|            children|              format|pages|
+--------------------+------------------+--------------------+----------+-----------+--------------------+--------------------+-----+
|Lists of men-of-w...|  nla.obj-33236945|https://trove.nla...|    1935.0|     1938.0|    nla.obj-33239056|Book|Book/Illustr...|  0.0|
|[Bessie Campbell,...|  nla.obj-39267340|https://trove.nla...|    1889.0|     1918.0|nla.obj-149626788...|                Book|  0.0|
|[Bessie Campbell,...|  nla.obj-39267340|https://trove.nla...|    1889.0|     1889.0|nla.obj-149626788...|                Book|  0.0|
|Report of the pro...| nla.obj-475664029|https://trove.nla...|    1860.0|     1860.0|nla.obj-475664242...|Government public...|  0.0|
|Westminster Abbey...| nla.obj-410740363|https://trove.nla...|

In [148]:
df_selected_ch_pd = df_selected_ch.toPandas()
df_selected_ch_pd

,title,trove_id,url,date_start,date_finish,children,format,pages
0,"Lists of men-of-war, 1650-1700",nla.obj-33236945,https://trove.nla.gov.au/work/13097671,1935.0,1938.0,nla.obj-33239056,Book|Book/Illustrated,0.0
1,"[Bessie Campbell, Banjo Queen : a collection o...",nla.obj-39267340,https://trove.nla.gov.au/work/14080449,1889.0,1918.0,nla.obj-1496267881|nla.obj-1496153359,Book,0.0
2,"[Bessie Campbell, Banjo Queen : a collection o...",nla.obj-39267340,https://trove.nla.gov.au/work/237278882,1889.0,1889.0,nla.obj-1496267881|nla.obj-1496153359,Book,0.0
3,Report of the proceedings of the Queensland Go...,nla.obj-475664029,https://trove.nla.gov.au/work/18245206,1860.0,1860.0,nla.obj-475664242|nla.obj-69105887,Government publication|Book,0.0
4,Westminster Abbey : a service of memorial and ...,nla.obj-410740363,https://trove.nla.gov.au/work/12422565,1968.0,1968.0,nla.obj-411080022|nla.obj-365300304,Book,0.0
...,...,...,...,...,...,...,...,...
222,William Caxton : a contribution in commemorati...,nla.obj-402484730,https://trove.nla.gov.au/work/18645810,1871.0,1871.0,nla.obj-402485076|nla.obj-75799669,Book|Book/Illustrated,0.0
223,"Grammar, vocabulary, and notes of the Wangerri...",nla.obj-745264448,https://trove.nla.gov.au/work/8447025,1910.0,1919.0,nla.obj-745264677,Book/Illustrated|Government publication|Book,0.0
224,Seii ryakuron / Gasshin-shi cho ; Kan Mozai do...,nla.obj-58419108,https://trove.nla.gov.au/work/15814183,1858.0,1858.0,nla.obj-58419119,Book|Book/Illustrated,0.0
225,Izesuq oko moloko imolaqu,nla.obj-1385777973,https://trove.nla.gov.au/work/237411791,1971.0,1971.0,nla.obj-1385790149|nla.obj-1385790278|nla.obj-...,Book,0.0


In [149]:
ro2 = session.execute("SELECT * FROM books_keyspace.parent where date_start<2000 and date_finish<2000  ALLOW FILTERING" )
l2 = []
for row in ro2:
    l2.append((row.title, row.trove_id, row.url, row.date_start, row.date_finish,row.parent, row.format,row.pages) )
    


In [150]:

rdd_selected_p = spark.sparkContext.parallelize(l2)

df_selected_p = rdd_selected_p.toDF(['title','trove_id',
 'url',
 'date_start',
 'date_finish',
    
    'parent',
    'format',
    'pages'])

df_selected_p.show()

+--------------------+------------------+--------------------+----------+-----------+------------------+--------------------+-----+
|               title|          trove_id|                 url|date_start|date_finish|            parent|              format|pages|
+--------------------+------------------+--------------------+----------+-----------+------------------+--------------------+-----+
|      Kangxi zi dian|  nla.obj-56155448|https://trove.nla...|    1716.0|     1716.0|  nla.obj-56116183|                Book|600.0|
|      Kangxi zi dian|  nla.obj-56180948|https://trove.nla...|    1716.0|     1716.0|  nla.obj-56116183|                Book|600.0|
|Zhong xiao jie yi...|  nla.obj-57729010|https://trove.nla...|    1800.0|     1800.0|  nla.obj-57728987|                Book| 64.0|
|      Kangxi zi dian|  nla.obj-56186140|https://trove.nla...|    1716.0|     1716.0|  nla.obj-56116183|                Book|600.0|
|      Kangxi zi dian|  nla.obj-56155644|https://trove.nla...|    1716.0|   

In [151]:
df_selected_p_pd = df_selected_p.toPandas()
df_selected_p_pd

,title,trove_id,url,date_start,date_finish,parent,format,pages
0,Kangxi zi dian,nla.obj-56155448,https://trove.nla.gov.au/work/237411458,1716.0,1716.0,nla.obj-56116183,Book,600.0
1,Kangxi zi dian,nla.obj-56180948,https://trove.nla.gov.au/work/237411458,1716.0,1716.0,nla.obj-56116183,Book,600.0
2,Zhong xiao jie yi er du mei zhuan,nla.obj-57729010,https://trove.nla.gov.au/work/12608633,1800.0,1800.0,nla.obj-57728987,Book,64.0
3,Kangxi zi dian,nla.obj-56186140,https://trove.nla.gov.au/work/237411458,1716.0,1716.0,nla.obj-56116183,Book,600.0
4,Kangxi zi dian,nla.obj-56155644,https://trove.nla.gov.au/work/237411458,1716.0,1716.0,nla.obj-56116183,Book,600.0
...,...,...,...,...,...,...,...,...
1342,Kangxi zi dian,nla.obj-56173146,https://trove.nla.gov.au/work/237411458,1716.0,1716.0,nla.obj-56116183,Book,600.0
1343,[Australian grocery shop advertisements of the...,nla.obj-2376117303,https://trove.nla.gov.au/work/238079962,1929.0,1935.0,nla.obj-2373353246,Book,1.0
1344,Izesuq oko moloko imolaqu,nla.obj-1385752809,https://trove.nla.gov.au/work/237411797,1971.0,1971.0,nla.obj-1385745492,Book,4.0
1345,Sōmoku zusetsu. Inuma Yokusai cho,nla.obj-60720863,https://trove.nla.gov.au/work/12611535,1856.0,1907.0,nla.obj-60720830,Book/Illustrated|Book,67.0


In [140]:
row = session.execute("SELECT avg(pages) FROM books_keyspace.parent" ).all()
value = row[0].system_avg_pages
print(value)

298.45257568359375


In [139]:
row = session.execute("SELECT avg(pages) FROM books_keyspace.children" ).all()
value = row[0].system_avg_pages
print(value)

0.0
